In [ ]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from blitz.modules import BayesianLSTM
from blitz.utils import variational_estimator

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
%matplotlib inline

from collections import deque

In [ ]:
!cd data && unzip stock-time-series-20050101-to-20171231

In [ ]:
"""
REMEMBER TO DOWNLOAD THE DATASET AT KAGGLE AT THIS LINK:
https://www.kaggle.com/szrlee/stock-time-series-20050101-to-20171231
"""
amazon="data/AMZN_2006-01-01_to_2018-01-01.csv"
ibm="data/IBM_2006-01-01_to_2018-01-01.csv"
df = pd.read_csv(ibm)

window_size = 21

In [ ]:

df.head()